#Week 3

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("RetailSalesInsights").getOrCreate()

In [2]:
from google.colab import files
uploaded = files.upload()


Saving cleaned_sales_data.csv to cleaned_sales_data.csv


In [3]:
import pandas as pd

df_check = pd.read_csv("cleaned_sales_data.csv")
print("Unique categories in uploaded file:", df_check['category'].unique())
print(df_check.head(10))
print(df_check.tail(10))


Unique categories in uploaded file: ['Electronics' 'Clothing' 'Furniture']
   sale_id   sale_date  quantity product_name     category    price     cost  \
0        1  2025-07-01         3       Laptop  Electronics  54000.0  48000.0   
1        2  2025-07-01        10        Mouse  Electronics    700.0    500.0   
2        4  2025-07-02        15   Headphones  Electronics   1500.0   1000.0   
3        5  2025-07-02        20        Shirt     Clothing   1200.0    800.0   
4        6  2025-07-03        10        Shoes     Clothing   2500.0   1800.0   
5        7  2025-07-03         3         Sofa    Furniture  20000.0  15000.0   

   store_name region employee_name             role   revenue   profit  \
0  All things  South   Rohit Kumar  Sales Associate  162000.0  18000.0   
1  Delhi Mall  North       Ankitha          Cashier    7000.0   2000.0   
2  All things  South   Rohit Kumar  Sales Associate   22500.0   7500.0   
3  Delhi Mall  North       Ankitha          Cashier   24000.0   8000

In [4]:
from pyspark.sql.functions import col, month, avg
from pyspark.sql.types import DoubleType
df = spark.read.csv("cleaned_sales_data.csv", header=True, inferSchema=True)
df.show()

underperforming = df.filter((col("quantity") < 5) | (col("profit") < 1000))
underperforming.select("sale_id", "product_name", "quantity", "profit").show()
df_with_month = df.withColumn("month", month(col("sale_date")))
monthly_avg = df_with_month.groupBy("store_name", "month").agg(avg("revenue").alias("avg_monthly_revenue"))
monthly_avg.show()
underperforming.toPandas().to_csv("underperforming_products.csv", index=False)
monthly_avg.toPandas().to_csv("monthly_avg_revenue.csv", index=False)



+-------+----------+--------+------------+-----------+-------+-------+----------+------+-------------+---------------+--------+-------+------------------+
|sale_id| sale_date|quantity|product_name|   category|  price|   cost|store_name|region|employee_name|           role| revenue| profit|     profit_margin|
+-------+----------+--------+------------+-----------+-------+-------+----------+------+-------------+---------------+--------+-------+------------------+
|      1|2025-07-01|       3|      Laptop|Electronics|54000.0|48000.0|All things| South|  Rohit Kumar|Sales Associate|162000.0|18000.0| 11.11111111111111|
|      2|2025-07-01|      10|       Mouse|Electronics|  700.0|  500.0|Delhi Mall| North|      Ankitha|        Cashier|  7000.0| 2000.0| 28.57142857142857|
|      4|2025-07-02|      15|  Headphones|Electronics| 1500.0| 1000.0|All things| South|  Rohit Kumar|Sales Associate| 22500.0| 7500.0| 33.33333333333333|
|      5|2025-07-02|      20|       Shirt|   Clothing| 1200.0|  800.0|